In [1]:
!pip install moviepy pydub


  Using cached python_dotenv-1.1.0-py3-none-any.whl.metadata (24 kB)
   ---------------------------------------- 0.0/31.2 MB ? eta -:--:--
   - -------------------------------------- 0.8/31.2 MB 3.7 MB/s eta 0:00:09
   -- ------------------------------------- 1.8/31.2 MB 5.0 MB/s eta 0:00:06
   --- ------------------------------------ 2.9/31.2 MB 4.8 MB/s eta 0:00:06
   ----- ---------------------------------- 3.9/31.2 MB 5.0 MB/s eta 0:00:06
   ------ --------------------------------- 5.2/31.2 MB 5.0 MB/s eta 0:00:06
   -------- ------------------------------- 6.3/31.2 MB 5.1 MB/s eta 0:00:05
   --------- ------------------------------ 7.6/31.2 MB 5.2 MB/s eta 0:00:05
   ---------- ----------------------------- 8.4/31.2 MB 5.3 MB/s eta 0:00:05
   ------------ --------------------------- 9.4/31.2 MB 5.0 MB/s eta 0:00:05
   ------------- -------------------------- 10.2/31.2 MB 5.0 MB/s eta 0:00:05
   ------------- -------------------------- 10.2/31.2 MB 5.0 MB/s eta 0:00:05
   ---------

  You can safely remove it manually.


In [2]:
!pip install tqdm

In [17]:
!pip install pytesseract


In [41]:
import subprocess
from pydub import AudioSegment, silence
import os
from tqdm import tqdm
from pathlib import Path
import os

# 1. Extrair o áudio usando ffmpeg
def extrair_audio_com_ffmpeg(video_path, audio_path):
    comando = [
        "ffmpeg",
        "-i", video_path,
        "-vn",              
        "-acodec", "pcm_s16le",  
        "-ar", "16000",     
        "-ac", "1",         
        audio_path
    ]
    subprocess.run(comando, check=True)

# 2. Dividir o áudio nos silêncios E registrar tempos
def dividir_audio(audio_path, output_audio_folder, video_path, output_image_folder, min_silence_len=300, silence_thresh=-40):
    audio = AudioSegment.from_wav(audio_path)

    os.makedirs(output_audio_folder, exist_ok=True)
    os.makedirs(output_image_folder, exist_ok=True)

    # Identifica os trechos de áudio entre silêncios
    silent_ranges = silence.detect_silence(
        audio,
        min_silence_len=min_silence_len,
        silence_thresh=silence_thresh
    )

    # Transformar a lista para começar e terminar onde há som
    boundaries = [0] + [item for sublist in silent_ranges for item in sublist] + [len(audio)]

    # Pega os intervalos de fala
    for i in tqdm(range(0, len(boundaries)-1, 2)):
        start = boundaries[i]
        end = boundaries[i+1]
        palavra = audio[start:end]

        # Salva o trecho de áudio
        palavra.export(f"{output_audio_folder}/palavra_{i//2 + 1}.wav", format="wav")

        # Calcula tempo em segundos
        tempo_medio_ms = (start + end) / 2
        tempo_medio_seg = tempo_medio_ms / 1000

        # Extrai a imagem do vídeo nesse tempo
        salvar_frame(video_path, tempo_medio_seg, f"{output_image_folder}/imagem_{i//2 + 1}.jpg")

# 3. Salvar um frame do vídeo em um tempo específico
def salvar_frame(video_path, tempo_segundos, output_imagem_path):
    comando = [
        "ffmpeg",
        "-ss", str(tempo_segundos),
        "-i", video_path,
        "-frames:v", "1",
        "-q:v", "2",  
        output_imagem_path
    ]
    subprocess.run(comando, check=True)

# 4. Exemplo de uso
def extract_data_from_videos():
    # Caminhos base
    video_dir = Path("data/video")
    audio_template = "data/audio/{topico}/{sub_topico}/audio_extraido.wav"
    output_audio_template = "data/audio/extract_to_video/{topico}/{sub_topico}"
    output_image_template = "data/image/extract_to_video/{topico}/{sub_topico}"

    # Itera sobre vídeos
    for video_file in video_dir.glob("*.mp4"):
        try:
            # Extrai nome sem extensão e separa tópicos
            video_name = video_file.stem.replace(".mp4", "")  # remove .mp4
            topico, sub_topico = video_name.split("__")

            print(f"🎥 Processando vídeo: {video_file.name}")
            print(f"📁 Tópico: {topico} | Subtópico: {sub_topico}")

            # Prepara caminhos
            audio_path = Path(audio_template.format(topico=topico, sub_topico=sub_topico))
            output_audio_folder = Path(output_audio_template.format(topico=topico, sub_topico=sub_topico))
            output_image_folder = Path(output_image_template.format(topico=topico, sub_topico=sub_topico))

            # Cria pastas de saída se não existirem
            output_audio_folder.mkdir(parents=True, exist_ok=True)
            output_image_folder.mkdir(parents=True, exist_ok=True)

            # Processa
            # extrair_audio_com_ffmpeg(video_file, audio_path)
            # dividir_audio(audio_path, output_audio_folder, video_file, output_image_folder)

        except Exception as e:
            print(f"❌ Erro ao processar {video_file.name}: {e}")

    print("✅ Finalizado!")

extract_data_from_videos()

🎥 Processando vídeo: topico_1__sub_topico_1.mp4
📁 Tópico: topico_1 | Subtópico: sub_topico_1
🎥 Processando vídeo: topico_2__sub_topico_2.mp4.mp4
📁 Tópico: topico_2 | Subtópico: sub_topico_2
✅ Finalizado!


---

In [ ]:
import cv2
import json
from pathlib import Path

def save_json_rects_to_crop():
    retangulos = []
    ponto_inicial = None

    # Caminho da imagem
    caminho_imagem = Path("data/palavras_extraidas_imagem/imagem_36.jpg")
    nome_imagem = caminho_imagem.stem
    output_json = caminho_imagem.with_name(f"retangulos_corte_imagem.json")

    # Carrega imagem e redimensiona
    img = cv2.imread(str(caminho_imagem))
    if img is None:
        print(f"❌ Erro ao carregar imagem: {caminho_imagem}")
        return

    escala = 0.4
    img_mini = cv2.resize(img, (int(img.shape[1] * escala), int(img.shape[0] * escala)))

    # Callback de clique
    def clique(event, x, y, flags, param):
        nonlocal ponto_inicial
        if event == cv2.EVENT_LBUTTONDOWN:
            if ponto_inicial is None:
                ponto_inicial = (x, y)
                print(f"🟢 Primeiro ponto: {ponto_inicial}")
            else:
                ponto_final = (x, y)
                print(f"🔴 Segundo ponto: {ponto_final}")
                x1, y1 = ponto_inicial
                x2, y2 = ponto_final
                retangulos.append((x1, y1, x2, y2))
                cv2.rectangle(img_mini, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.imshow("Imagem", img_mini)
                ponto_inicial = None

    # Interface gráfica
    cv2.imshow("Imagem", img_mini)
    cv2.setMouseCallback("Imagem", clique)
    print("🖱️ Clique dois pontos para cada retângulo (canto sup. esquerdo e inf. direito). Pressione 'q' para sair.")

    while True:
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cv2.destroyAllWindows()

    # Salva os retângulos no JSON
    with open(output_json, "w") as f:
        json.dump(retangulos, f)
    print(f"✅ {len(retangulos)} retângulo(s) salvo(s) em {output_json}")

# Exemplo de uso
if __name__ == "__main__":
    save_json_rects_to_crop()


🖱️ Clique dois pontos para cada retângulo (canto sup. esquerdo e inf. direito). Pressione 'q' para sair.
🟢 Primeiro ponto: (274, 358)


In [80]:
import cv2
import json
from pathlib import Path
import numpy as np

# Parâmetros globais
ESCALA = 0.4
PASTA_IMAGENS = Path("data/image/extract_to_video")
PASTA_JSON_RETANGULOS_CORTE = Path("data/json/retangulos_corte_imagem.json")
PASTA_SAIDA = Path("data/recortes")

# OCR futuro
def aplicar_ocr_em_imagem(imagem_path):
    # Substitua por pytesseract ou outro OCR
    return f"[Texto detectado aqui da {imagem_path}]"

def save_json(json_path, dict_text):
    # Salva o dicionário como JSON
    with open(json_path, "w", encoding="utf-8") as f:
        json.dump(dict_text, f, indent=2, ensure_ascii=False)
        
def resize_frame(frame, escala=ESCALA):
    altura, largura = frame.shape[:2]
    nova_largura = int(largura * escala)
    nova_altura = int(altura * escala)
    return cv2.resize(frame, (nova_largura, nova_altura))

def processar_imagem(caminho_imagem: Path):
    nome_arquivo = caminho_imagem.stem  # Ex: imagem_42
    json_path = PASTA_JSON_RETANGULOS_CORTE

    if not json_path.exists():
        print(f"⚠️ JSON não encontrado para {caminho_imagem.name}")
        return

    img = cv2.imread(str(caminho_imagem))
    if img is None:
        print(f"❌ Erro ao carregar imagem: {caminho_imagem}")
        return

    with open(json_path, "r") as f:
        retangulos = json.load(f)

    if not retangulos:
        print(f"❌ Nenhum retângulo em {json_path}")
        return

    # Caminho de saída preservando a estrutura
    estrutura_relativa = caminho_imagem.relative_to(PASTA_IMAGENS).parent
    pasta_destino = PASTA_SAIDA / estrutura_relativa / nome_arquivo
    pasta_destino.mkdir(parents=True, exist_ok=True)

    # Redimensiona imagem
    # img_resize = resize_frame(img)
    dict_text = {}

    for i, (x1, y1, x2, y2) in enumerate(retangulos):
        x1, x2 = sorted([x1, x2])
        y1, y2 = sorted([y1, y2])

        # Corrige coordenadas com base na escala original
        x1 = int(x1 / ESCALA)
        x2 = int(x2 / ESCALA)
        y1 = int(y1 / ESCALA)
        y2 = int(y2 / ESCALA)

        recorte = img[y1:y2, x1:x2]

        image_name = f"recorte_{i}.jpg"
        caminho_recorte = os.path.join(pasta_destino, image_name)
        
        cv2.imwrite(str(caminho_recorte), recorte)
        print(f"✅ Recorte salvo: {caminho_recorte}")

        if i != 0:
            texto = aplicar_ocr_em_imagem(caminho_recorte)
            dict_text.update({image_name:texto})
            print(f"📝 Texto extraído: {texto}")
        
        
   # Ao final, o dict_text contém os textos dos recortes (exceto o primeiro)
    print("📄 Dicionário final de textos extraídos:")
    json_path = os.path.join(pasta_destino, "text.json")
    save_json(json_path=json_path, dict_text=dict_text)

# Procura todas as imagens dentro dos tópicos/subtópicos
for caminho_imagem in PASTA_IMAGENS.rglob("*.jpg"):
    processar_imagem(caminho_imagem)


✅ Recorte salvo: data\recortes\topico_1\sub_topico_1\imagem_1\recorte_0.jpg
✅ Recorte salvo: data\recortes\topico_1\sub_topico_1\imagem_1\recorte_1.jpg
📝 Texto extraído: [Texto detectado aqui da data\recortes\topico_1\sub_topico_1\imagem_1\recorte_1.jpg]
✅ Recorte salvo: data\recortes\topico_1\sub_topico_1\imagem_1\recorte_2.jpg
📝 Texto extraído: [Texto detectado aqui da data\recortes\topico_1\sub_topico_1\imagem_1\recorte_2.jpg]
✅ Recorte salvo: data\recortes\topico_1\sub_topico_1\imagem_1\recorte_3.jpg
📝 Texto extraído: [Texto detectado aqui da data\recortes\topico_1\sub_topico_1\imagem_1\recorte_3.jpg]
✅ Recorte salvo: data\recortes\topico_1\sub_topico_1\imagem_1\recorte_4.jpg
📝 Texto extraído: [Texto detectado aqui da data\recortes\topico_1\sub_topico_1\imagem_1\recorte_4.jpg]
📄 Dicionário final de textos extraídos:
✅ Recorte salvo: data\recortes\topico_1\sub_topico_1\imagem_10\recorte_0.jpg
✅ Recorte salvo: data\recortes\topico_1\sub_topico_1\imagem_10\recorte_1.jpg
📝 Texto extraí

In [87]:
(15 * 5) * 2

150

In [ ]:
(0.33 * 15) #gasto (valor * minutos_usados)

4.95

In [78]:
list_text = {}

list_text.update({"teste":"ok"})
list_text.update({"teste2":"ok"})


list_text

{'teste': 'ok', 'teste2': 'ok'}

In [62]:
import numpy as np

In [66]:
type(img)

numpy.ndarray

True

In [52]:
caminho_imagem = r"data\image\extract_to_video\topico_1\sub_topico_1\imagem_42.jpg"
img = cv2.imread(caminho_imagem)

if img is None:
    print("❌ Nenhum imagem encontrada.")